**Agenda**: To build an AI agent with persistence memory using FilesystemBackend (using Langchain DeepAgent access to local file system to store the data)

**Install packages**

In [ ]:
!pip install -qU deepagents
!pip install -U "langchain[openai]"

**Import packages**

In [ ]:
import os
from langchain.chat_models import init_chat_model
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend

**Set OpenAI API key**

In [ ]:
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

**Building Agent**

Creating deep agent with backend option FilesystemBackend (giving access to local file system using absolute path)
* I have created a root folder called memories and in the system prompt I have explicity mentioned to store demographic information under user.txt and preferences in user_preferences.txt file.

In [ ]:
agent = create_deep_agent(
    model = init_chat_model("openai:gpt-4.1-mini"),
    backend=FilesystemBackend(root_dir="/content/memories"),
    system_prompt = """
You are a memory-enabled assistant.

Your job is to store and retrieve user information clearly and concisely.

--------------------------------------------------
MEMORY LOCATION
--------------------------------------------------

All memory is stored under:

/content/memories/

Files:

1) /content/memories/user.txt
   Store demographic information as clear sentences.
   Example:
   - Adam is 51 years old.
   - Adam is unmarried.
   - Adam was born on June 10, 1973.

2) /content/memories/user_preferences.txt
   Store user preferences as clear sentences.
   Example:
   - Adam prefers short responses.
   - Adam likes jogging twice a day.

--------------------------------------------------
STORAGE RULES
--------------------------------------------------

- Before saving, read the existing file.
- If the exact sentence already exists, do NOT save again.
- If it is new information, append it.
- When storing:
    Return exactly:
    "Data stored."
    or
    "Data already stored."

--------------------------------------------------
RETRIEVAL RULES
--------------------------------------------------

- When the user asks a question about stored information:
    • Search the relevant file.
    • Return ONLY the exact stored sentence that answers the question.
    • Do NOT rephrase.
    • Do NOT explain.
    • Do NOT add extra words.

- If no relevant information exists:
    Return exactly:
    "No stored data."

--------------------------------------------------
GENERAL RULES
--------------------------------------------------

- Keep responses minimal.
- Do not expose file paths.
- Do not fabricate information.
"""
)

**Asking agent to store a preference**

In [ ]:
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "adam likes jogging twice a day",
            }
        ]
    }
)

print(result['messages'][-1].content)

Data stored.


**Testing agent to retrieve earlier stored preference**

In [ ]:
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "list all preferences of adam?",
            }
        ]
    }
)

print(result['messages'][-1].content)

Adam likes jogging twice a day.


Agent was able to successfully retrieve the saved preference from the local file.

**Now lets ask agent to save the demographics of the user**

In [ ]:
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "I am Adam and i am 51 years old, unmarried and sex is male",
            }
        ]
    }
)

print(result['messages'][-1].content)

Data stored.


**Ask agent to fetch both demographics and preferences**

In [ ]:
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "how old is adam and what are his preferences?",
            }
        ]
    }
)

print(result['messages'][-1].content)

Adam is 51 years old.
Adam likes jogging twice a day.


Agent is successful in getting answers after looking at data stored in both user.txt and user_preferences.txt files to answer the query

**Conclusion**

Agents can use a local file system to store user data, enabling persistent memory and supporting the development of long term memory agents.